# Análisis e Implementación en SparkQL

* Entrega hasta el 24 de mayo en https://forms.gle/aRiWtjayausRFCTd8

* Los dos conjuntos de datos entregados, CSV separados por coma, Evaluación del Pitch (2021.04.13 ISoftware).csv y Asistencia del Pitch (2021.04.13 ISoftware).csv proceden de dos encuestas realizadas con Google Forms.

* Los archivos proceden de las evaluaciones de los pitch (exposiciones cortas de negocio) que realizan los estudiantes de la asignatura de Ingeniería de Software. Los estudiantes están organizados en equipos. Cada equipo realiza una presentación de máximo 15 minutos, una vez inicia la presentación todos los estudiantes deben registrar su asistencia lo cual queda registrado en 'Asistencia del Pitch (2021.04.13 ISoftware).csv' incluidos los miembros del equipo.

* El archivo `Asistencia del Pitch (2021.04.13 ISoftware).csv` contiene las columnas: `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto,`"Equipo al que perteneces:"` equipo de trabajo al que pertenece el estudiante también es texto y `"Equipo que va a exponer:"` equipo que el estudiante va a tender a su presentación.

* El archivo `Evaluación del Pitch (2021.04.13 ISoftware).csv` contiene las columnas `"Marca temporal"` es tiempo dado en fecha y hora, `"Nombre de usuario"` es el correo electrónico del estudiante y es texto, `"Equipo que vas a evaluar:"` equipo que ha expuesto y que va a ser evaluado por cada estudiante que no sea integrante; a continuación, se tienen las siguientes columnas que corresponden a la evaluación de los respectivos ítems:

    1. "Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?",
    2. "Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?",
    3. "Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?",
    4. "Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?",
    5. "Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?",
    6. "Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.",
    7. "Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.",
    8. "Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.",
    9. "Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?",
    10. "Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? ",
    11. "Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?",
    12. "Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.",
    13. "Exposición: ¿Qué tan coordinados estaban los expositores?",
    14. "Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?",
    15. "Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.",
    * "Suponiendo que eres inversionista, ¿Estarías dispuesto a invertir dinero en este equipo? (esta pregunta no se pondera en la nota)",
    * "Observaciones para el equipo, estas observaciones las debe considerar el equipo para mejorar la siguiente presentación."


* Cada ítem se evalúa con la siguiente escala: 0. Ausente; 1. Deficiente; 2. Regular; 3. Aceptable; 4. Bueno; 5. Excelente

* Carge los datos, cada archivo en una tabla SparkSQL y responda cada una de las consultas dadas en cada celda. Tenga en cuenta que algunas consultas pueden tener como resultado el vacío.

# Integrantes del equipo
1. Simón Zapata Gutiérrez

In [ ]:
# Agrege acá el código para importar las librerias
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from  pyspark.sql.functions import concat_ws
from pyspark.sql.functions import to_timestamp
from  pyspark.sql.functions import collect_list
from  pyspark.sql.functions import regexp_replace

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [ ]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell. 
# Tu código a continuación...

!hdfs dfs -copyFromLocal 'asistencia.csv' /tmp

In [ ]:
!hdfs dfs -copyFromLocal 'evaluacion.csv' /tmp

In [ ]:
!hadoop fs -ls /tmp

In [ ]:
# 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
# Observación: tenga especial cuidado con los encabezados de los archivos CSV.
# Usted puede considerar cambiar los encabezados de los CSV originales
# Tu código a continuación...

# Se carga archivo de asistencia
asistencia = ContextoSql.read.load('/tmp/asistencia.csv',
                                   format="csv",
                                   sep=',',
                                   inferSchema='true',
                                   header='true')

# Se carga archivo de evaluación
evaluacion = ContextoSql.read.load('/tmp/evaluacion.csv',
                                   format="csv",
                                   sep=',',
                                   inferSchema='true',
                                   header='true')

In [ ]:
asistencia.printSchema()

In [ ]:
evaluacion.printSchema()

In [ ]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
# Tu código a continuación...

evaluacion = evaluacion.filter(evaluacion["Nombre de usuario"].contains("@"))
# Se realiza trim de evaluacion
trimmed_evaluacion = trim(evaluacion['Nombre de usuario'])
evaluacion = evaluacion.withColumn('Nombre de usuario', trimmed_evaluacion)

evaluacion.select('Nombre de usuario').distinct().orderBy(
    'Nombre de usuario', ascending=True).show(10000)

In [ ]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...

# Conversión a formato fecha
asistencia = asistencia.withColumn(
    "Marca temporal", (to_timestamp(asistencia["Marca temporal"], 
                                    'yyyy/MM/dd HH:mm:ss')))

# Conteo asistencia
asistencia.groupBy("Nombre de usuario").agg(
    F.count('Nombre de usuario').alias("Número de asistencias"), 
    F.max("Marca temporal").alias("Última asistencia"),
    F.min("Marca temporal").alias("Primera asistencia")).orderBy(
    ["Número de asistencias","Nombre de usuario"], 
    ascending=[False,True]).show(3000)

In [ ]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...

# Grupo de asistencias
asistencias = asistencia.groupBy("Nombre de usuario").agg(
    F.count('Nombre de usuario').alias("Número de asistencias"), 
    F.max("Marca temporal").alias("Última asistencia"),
    F.min("Marca temporal").alias("Primera asistencia")).orderBy(
    ["Número de asistencias","Nombre de usuario"], ascending=[False, True])

# Listado de menos de 2 presentaciones
asistencias.withColumn(
    "Número de asistencias",
    asistencias["Número de asistencias"].cast(IntegerType())).where(
    asistencias['Número de asistencias']<=2).show(1000)

In [ ]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...

# Estudiantes que no asistieron a presentaciones
no_presentacion_est = evaluacion.select(
    'Nombre de usuario').distinct().orderBy(
    'Nombre de usuario', ascending=True)

no_presentacion_est.join(grupo_asistencias,
                         on=["Nombre de usuario"], 
                         how = 'left_anti').show(10)


In [ ]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...

# Reemplazos en columnas
integrantes_equipo = asistencia.withColumn(
    "Equipo al que perteneces:", 
    regexp_replace("Equipo al que perteneces:",
                   'Pertenezco a:', ''))

# Reemplazos en columnas
integrantes_equipo = integrantes_equipo.withColumn(
    "Equipo que va a exponer:", 
    regexp_replace("Equipo que va a exponer:", 'Expone: ', ""))

# Se envía a tabla en pandas
integrantes_equipo.groupby(
    "Equipo al que perteneces:").agg(concat_ws(
    ", ",collect_list("Nombre de usuario")).alias("Integrantes")).toPandas()

In [ ]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...

integrantes_equipo_2 = integrantes_equipo.withColumn(
    "Equipo que va a exponer:",
    trim(integrantes_equipo["Equipo que va a exponer:"]))

integrantes_equipo_2 = integrantes_equipo_2.withColumn(
    "Equipo al que perteneces:",
    trim(integrantes_equipo_2["Equipo al que perteneces:"]))

integrantes_equipo_2.where(
    integrantes_equipo_2["Equipo que va a exponer:"] != 
    integrantes_equipo_2["Equipo al que perteneces:"]).groupBy(
    "Equipo que va a exponer:").agg(
    F.count("Nombre de usuario").alias(
        "Asistentes")).orderBy(
    "Asistentes", ascending=False).show(1000)

In [ ]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...

evaluacion_propia = evaluacion.withColumn(
    'Equipo que vas a evaluar:', 
    trim(Evaluacion['Equipo que vas a evaluar:']))
evaluacion_propia = evaluacion_propia.select(
    'Nombre de usuario', 'Equipo que vas a evaluar:').distinct()


equipos = evaluacion_propia.join(integrantes_equipo_2.select(
    'Nombre de usuario', 'Equipo al que perteneces:').distinct(), 
    on=['Nombre de usuario'], 
    how='left')
equipos.filter(equipos['Equipo que vas a evaluar:'] == equipos['Equipo al que perteneces:']
              ).orderBy('Nombre de usuario').show(1000)

In [ ]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

evaluacion_equipo = evaluacion.withColumn('Equipo que vas a evaluar:', 
                                          trim(Evaluacion['Equipo que vas a evaluar:']))


evaluacion_propia = evaluacion_propia.select(
    'Nombre de usuario', 'Equipo que vas a evaluar:', 
    evaluacion_equipo['Introducción: El equipo responde adecuadamente ¿Quiénes son y por qué están aquí?'].alias('Introducción'),
    evaluacion_equipo['Equipo: El equipo responde adecuadamente ¿Quiénes están detrás de la idea y cuál es su función?'].alias('Equipo'),
    evaluacion_equipo['Problema: El equipo responde adecuadamente ¿Qué problema resolverá?, ¿es realmente un problema?'].alias('Problema'),
    evaluacion_equipo['Ventajas: El equipo responde adecuadamente ¿Por qué su solución es especial?, ¿qué la hace distinta de otras?'].alias('Ventajas'),
    evaluacion_equipo['Solución: El equipo responde adecuadamente ¿Cómo piensa resolver el problema?'].alias('Solución'),
    evaluacion_equipo["`Producto: El equipo responde adecuadamente ¿Cómo funciona el producto o servicio? Muestra algunos ejemplos.`"].alias('Producto'),
    evaluacion_equipo["`Tracción: El equipo responde adecuadamente si cuenta con clientes que demuestran potencial.`"].alias('Tracción'),
    evaluacion_equipo["`Mercado: El equipo responde conoce, o por lo menos intentar predecir, el tamaño del mercado que impactará.`"].alias('Mercado') ,
    evaluacion_equipo["`Competencia: El equipo responde adecuadamente ¿Cuáles son las soluciones alternativas al problema que plantea?`"].alias('Competencia'),
    evaluacion_equipo["`Modelo de negocio: El equipo responde adecuadamente ¿Cómo hará dinero? `"].alias('Modelo de negocio'),
    evaluacion_equipo["`Inversión: El equipo responde adecuadamente ¿Cuál es su presupuesto y cuánto espera ganar?`"].alias('Inversión'),
    evaluacion_equipo["`Contacto: El equipo deja los datos al cliente y muestra cómo pueden contactarle.`"].alias('Contacto'),
    evaluacion_equipo["`Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?`"].alias('Exposicion1'),
    evaluacion_equipo["`Exposición: ¿Los expositores se expresaron con claridad y se hicieron entender?`"].alias('Exposicion2'),
    evaluacion_equipo["`Exposición: Las diapositivas son claras y coherentes y apoyaron adecuadamente la exposición.`"].alias('Exposicion3'))

Equipos = evaluacion_equipo.join(
    integrantes_equipo_2.select(
        'Nombre de usuario', 'Equipo al que perteneces:').distinct(), 
    on=['Nombre de usuario'], 
    how='left')

evaluaciones = equipos.filter((equipos['Equipo que vas a evaluar:'] != 
                               equipos['Equipo al que perteneces:']))

promedios = evaluaciones.groupBy('Equipo que vas a evaluar:').agg(
    F.avg('Introducción'), F.avg('Equipo'), F.avg('Problema'), 
    F.avg('Ventajas'), F.avg('Solución'), F.avg('Producto'), 
    F.avg('Tracción'), F.avg('Mercado'), F.avg('Competencia'), 
    F.avg('Modelo de negocio'), F.avg('Inversión'), F.avg('Contacto'), 
    F.avg('Exposicion1'), F.avg('Exposicion2'), F.avg('Exposicion3'))

nota = promedios.select(
    '*', 
    ((promedios["`avg(Introducción)`"] + promedios["`avg(Equipo)`"] + 
      promedios["`avg(Problema)`"] + promedios["`avg(Ventajas)`"] + 
      promedios["`avg(Solución)`"] + promedios["`avg(Producto)`"] + 
      promedios["`avg(Tracción)`"] + promedios["`avg(Mercado)`"] + 
      promedios["`avg(Competencia)`"] + promedios["`avg(Modelo de negocio)`"] 
      + promedios["`avg(Inversión)`"] + promedios["`avg(Contacto)`"] +
      promedios["`avg(Exposicion1)`"] + promedios["`avg(Exposicion2)`"] + 
      promedios["`avg(Exposicion3)`"])/15).alias('Nota final'))

nota.toPandas()

In [ ]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

def consultar_mejor_equipo(colname):
    nota.where(nota[colname] == nota.agg({colname: "max"}).collect()[0][0]).select(
        'Equipo que vas a evaluar:',colname).show()

for col in nota.columns[1:]:
    consultar_mejor_equipo(col)

In [ ]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...


def consultar_peor_equipo(colname):
    nota.where(nota[colname] == nota.agg({colname: "min"}).collect()[0][0]).select(
        'Equipo que vas a evaluar:',colname).show()

for col in nota.columns[1:]:
    consultar_peor_equipo(col)

In [ ]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

integrantes_equipo_3 = integrantes_equipo.withColumn(
    'Equipo al que perteneces:', 
    trim(integrantes_equipo['Equipo al que perteneces:']))

integrantes_equipo_3.select(
    'Nombre de usuario', 
    'Equipo al que perteneces:').distinct().join(
    nota,
    on=(integrantes_equipo_3['Equipo al que perteneces:'] == nota['Equipo que vas a evaluar:']),
    how='left').select('Nombre de usuario', 'Equipo al que perteneces:', 'Nota final').show()